In [ ]:
!pip install kaggle

In [ ]:
!mkdir .p~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘.p~/.kaggle’: No such file or directory
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 78% 63.0M/80.9M [00:00<00:00, 225MB/s]
100% 80.9M/80.9M [00:00<00:00, 222MB/s]


In [ ]:
 from zipfile import ZipFile
 file_name = "/content/sentiment140.zip"
 with ZipFile(file_name,'r') as zip:
   zip.extractall()
   print('done')

done


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding ='ISO.8859.1' )

In [ ]:
 twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
column_names=['target','ids','date','flag','user','text']
twitter_data.columns=column_names

In [ ]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding ='ISO.8859.1' ,names=column_names)

In [ ]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:
lemmatizer = WordNetLemmatizer()
def preprocess_text(content):
    # Remove special characters, numbers, and punctuation
    content = re.sub(r'[^a-zA-Z]', ' ', content)
    # Convert to lowercase and tokenize
    content = content.lower().split()
    # Remove stopwords and lemmatize
    content = [lemmatizer.lemmatize(word) for word in content if word not in stopwords.words('english')]
    return ' '.join(content)


In [ ]:
twitter_data['text']=twitter_data['text'].apply(preprocess_text)

In [ ]:
twitter_data.head()


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset update facebook texting might cry result...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan dived many time ball managed save res...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass behaving mad see


In [ ]:
print(twitter_data['text'])
print(twitter_data['target'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset update facebook texting might cry result...
2          kenichan dived many time ball managed save res...
3                            whole body feel itchy like fire
4                           nationwideclass behaving mad see
                                 ...                        
1599995                        woke school best feeling ever
1599996    thewdb com cool hear old walt interview http b...
1599997                       ready mojo makeover ask detail
1599998    happy th birthday boo alll time tupac amaru sh...
1599999    happy charitytuesday thenspcc sparkscharity sp...
Name: text, Length: 1600000, dtype: object
0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X=twitter_data['text']
Y=twitter_data['target']

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.30,stratify=Y,random_state=2)
print(X.shape)
print(X_train.shape)
print(X_test.shape)
print(Y.shape)
print(Y_train.shape)
print(Y_test.shape)
print(X_train)
print(Y_train)

(1600000,)
(1120000,)
(480000,)
(1600000,)
(1120000,)
(480000,)
803269                     finished reading book second time
1338796    tommyguns back action lunch today best watchou...
1480241                              kata shall listen later
430643                  jes got text someone even tried call
1367427                             ddlovatocyrus ok excited
                                 ...                        
207009     awwww zoo experience taken tear http yfrog com...
1520139    finally getting around watching season premier...
1589634                      jedbramwell thanks teddytuesday
1204636    mtv movie award hope rob win really care twili...
526253                  love waking folgers bad voice deeper
Name: text, Length: 1120000, dtype: object
803269     1
1338796    1
1480241    1
430643     0
1367427    1
          ..
207009     0
1520139    1
1589634    1
1204636    1
526253     0
Name: target, Length: 1120000, dtype: int64


In [ ]:
vectorizer=TfidfVectorizer(max_features=20000,min_df=5,max_df=0.7)
vectorizer.fit(X_train)
X_train=vectorizer.transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
print(X_train)
print(X_test)


  (0, 1997)	0.45828033255030287
  (0, 6181)	0.459243985422611
  (0, 14075)	0.4819748006861741
  (0, 15239)	0.5044884416809892
  (0, 17641)	0.303752863961067
  (1, 145)	0.47859808344380056
  (1, 1236)	0.24458741036928563
  (1, 1477)	0.41544207565123853
  (1, 1645)	0.31277348509094643
  (1, 2102)	0.35146534710824096
  (1, 10382)	0.35541953605945864
  (1, 17137)	0.36937655163756755
  (1, 17745)	0.23326278804199377
  (2, 9748)	0.5020689734058481
  (2, 10081)	0.5950393007703081
  (2, 15418)	0.6275786615891116
  (3, 2558)	0.33363875588765257
  (3, 5672)	0.29575372767637637
  (3, 7114)	0.23338656692010928
  (3, 8992)	0.6054003733819893
  (3, 16130)	0.32263086640788646
  (3, 17372)	0.3744937450996177
  (3, 18022)	0.36864586034244246
  (4, 5747)	0.7354031254535596
  (4, 12282)	0.6776298717390903
  :	:
  (1119996, 7634)	0.363403652097753
  (1119996, 8624)	0.2502748087775153
  (1119996, 13502)	0.34258454715786135
  (1119996, 15224)	0.2552828805931744
  (1119996, 16782)	0.2113729330662779
  (11199

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
params = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga']
}

grid = GridSearchCV(LogisticRegression(max_iter=2000, class_weight='balanced'), param_grid=params, cv=5, scoring='accuracy')
grid.fit(X_train, Y_train)

# Get the best model
model = grid.best_estimator_
print("Best Parameters:", grid.best_params_)


Best Parameters: {'C': 1, 'penalty': 'l2', 'solver': 'saga'}


In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train, X_train_prediction)
print(training_data_accuracy)


0.7856589285714286


In [ ]:
X_test_prediciton=model.predict(X_test)
test_data_accuracy=accuracy_score(Y_test,X_test_prediciton)
print(test_data_accuracy)

0.7773770833333333


how to save model


In [ ]:
import pickle

In [ ]:
filename='twitter_model.sav'
pickle.dump(model,open(filename,'wb'))

how to use saved model for future

In [ ]:
loaded_model=pickle.load(open(filename,'rb'))

using saved model

In [ ]:
loaded_model= pickle.load(open('/content/twitter_model.sav','rb'))

In [ ]:
X_new=X_test[30]
prediction=loaded_model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('negative')
else:
  print('positive')

[1]
positive
